# Ejercicios de APIs

En este notebook consolidamos los ejercicios de APIs, refactorizados para funcionar mejor en un entorno interactivo y guardar los resultados organizadamente.

## 1. OpenWeatherMap

Este ejercicio consulta el clima de ciudades especificadas y guarda los resultados en `resultados/resultados_clima.json`.

In [1]:
import requests
import json
import os
from collections import Counter

def consultar_clima(ciudades):
    api_key = 'db815c2f53d68f9b5124dc2116eb73b1'
    resultados = []
    temperaturas = []
    descripciones = []

    print("Iniciando consultas de clima...")
    
    for ciudad in ciudades:
        url = f'http://api.openweathermap.org/data/2.5/weather?q={ciudad}&appid={api_key}&units=metric'
        try:
            respuesta = requests.get(url)
            if respuesta.status_code == 200:
                clima = respuesta.json()
                descripcion = clima["weather"][0]["description"]
                temperatura = clima["main"]["temp"]

                resultados.append({ciudad: {"descripcion": descripcion, "temperatura": temperatura}})
                temperaturas.append(temperatura)
                descripciones.append(descripcion)

                print(f'Clima en {ciudad}: {descripcion}, Temperatura: {temperatura}°C')
            else:
                print(f"Error en {ciudad}: {respuesta.status_code} - {respuesta.json().get('message', 'Error desconocido')}")
        except requests.exceptions.RequestException as e:
            print(f"Error de conexión con {ciudad}: {e}")

    # Procesamiento de datos
    promedio = 0
    descripcion_mas_comun = ("N/A", 0)

    if temperaturas:
        promedio = sum(temperaturas) / len(temperaturas)
        print('-' * 50)
        print(f"Temperatura promedio: {round(promedio, 2)}°C")

    if descripciones:
        contador_descripciones = Counter(descripciones)
        descripcion_mas_comun = contador_descripciones.most_common(1)[0]
        print(f"Clima más común: '{descripcion_mas_comun[0]}' ({descripcion_mas_comun[1]} veces)")

    # Guardar resultados
    # Aseguramos que el directorio exista (aunque ya debería existir)
    os.makedirs('resultados', exist_ok=True)
    archivo_salida = 'resultados/resultados_clima.json'
    
    datos_a_guardar = {
        "resultados": resultados,
        "temperatura_promedio": promedio,
        "descripcion_mas_frecuente": {
            "descripcion": descripcion_mas_comun[0],
            "ocurrencias": descripcion_mas_comun[1]
        }
    }

    with open(archivo_salida, 'w') as archivo:
        json.dump(datos_a_guardar, archivo, indent=4)
    
    print(f"Resultados guardados en '{archivo_salida}'")

# Lista de ciudades de ejemplo para ejecutar
lista_ciudades = ["Montevideo", "Buenos Aires", "Madrid", "London", "Paris", "Tokyo", "New York"]
consultar_clima(lista_ciudades)

Iniciando consultas de clima...
Clima en Montevideo: few clouds, Temperatura: 26.73°C
Clima en Buenos Aires: clear sky, Temperatura: 28.42°C
Clima en Madrid: broken clouds, Temperatura: 8.95°C
Clima en London: scattered clouds, Temperatura: 6.74°C
Clima en Paris: clear sky, Temperatura: 12.29°C
Clima en Tokyo: clear sky, Temperatura: 5.64°C
Clima en New York: few clouds, Temperatura: 5.73°C
--------------------------------------------------
Temperatura promedio: 13.5°C
Clima más común: 'clear sky' (3 veces)
Resultados guardados en 'resultados/resultados_clima.json'


## 2. TheMovieDB

Este ejercicio busca películas y ahora guarda los resultados en `resultados/peliculas.json`.

In [2]:
import requests
import json
import os

api_key = '87c2a29f0f67d8cac27f27ab035b6a8b'
base_url = 'https://api.themoviedb.org/3'

def buscar_pelicula(titulo, idioma="en", anio=None, orden_popularidad="desc"):
    url = f"{base_url}/search/movie"
    parametros = {
        "api_key": api_key,
        "query": titulo,
        "language": idioma,
        "year": anio,
        "sort_by": f"popularity.{orden_popularidad}"
    }
    
    resultados_busqueda = []

    try:
        respuesta = requests.get(url, params=parametros)
        respuesta.raise_for_status()
        datos = respuesta.json()
        
        if datos["results"]:
            print(f"Resultados para '{titulo}':")
            for pelicula in datos["results"]:
                info = {
                    "titulo": pelicula['title'],
                    "fecha_lanzamiento": pelicula.get('release_date')
                }
                resultados_busqueda.append(info)
                print(f" - Título: {info['titulo']}, Fecha de lanzamiento: {info['fecha_lanzamiento']}")
            
            # Guardar en JSON
            os.makedirs('resultados', exist_ok=True)
            archivo_salida = 'resultados/peliculas.json'
            
            # Si queremos acumular resultados, primero leemos lo que hay (opcional), 
            # pero para este ejercicio sobrescribiremos o haremos append simple.
            # Aquí guardaremos solo la última búsqueda para simplicidad, o un diccionario de búsquedas.
            
            data_final = {
                "busqueda": titulo,
                "resultados": resultados_busqueda
            }

            with open(archivo_salida, 'w') as archivo:
                json.dump(data_final, archivo, indent=4)
            print(f"\nResultados guardados en '{archivo_salida}'")
            
        else:
            print("No se encontraron resultados para esa búsqueda.")

    except requests.exceptions.RequestException as e:
        print("Error al hacer la solicitud:", e)

# Ejecución de ejemplo
buscar_pelicula('Harry Potter')

Resultados para 'Harry Potter':
 - Título: Harry Potter and the Philosopher's Stone, Fecha de lanzamiento: 2001-11-16
 - Título: Harry Potter and the Half-Blood Prince, Fecha de lanzamiento: 2009-07-15
 - Título: Harry Potter and the Goblet of Fire, Fecha de lanzamiento: 2005-11-16
 - Título: Harry Potter and the Chamber of Secrets, Fecha de lanzamiento: 2002-11-13
 - Título: Harry Potter and the Prisoner of Azkaban, Fecha de lanzamiento: 2004-05-31
 - Título: Harry Potter and the Deathly Hallows: Part 2, Fecha de lanzamiento: 2011-07-12
 - Título: Harry Potter and the Deathly Hallows: Part 1, Fecha de lanzamiento: 2010-11-17
 - Título: Harry Potter and the Order of the Phoenix, Fecha de lanzamiento: 2007-07-08
 - Título: Harry Potter 20th Anniversary: Return to Hogwarts, Fecha de lanzamiento: 2022-01-01
 - Título: Harry Potter: A Spirit Conspiracy?, Fecha de lanzamiento: 2005-09-22
 - Título: Harry Potter: Witchcraft Repackaged, Fecha de lanzamiento: 2001-01-01
 - Título: Harry Potter